# Decile and Quartile Analysis

Analyzing data based on sample quantiles is another important tool for financial analysts. For example, the performance of a stock portfolio could be broken down into quartiles (four equal-sized chunks) based on each stock’s price-to-earnings. Using pan das.qcut combined with groupby makes quantile analysis reasonably straightforward.

As an example, let’s consider a simple trend following or momentum strategy trading the S&P 500 index via the SPY exchange-traded fund. You can download the price history from Yahoo! Finance:

In [45]:
from pandas_datareader import data
import pandas as pd
import numpy as np

In [46]:
dta = pd.read_csv('../../../CSV Files/O_Reilly/ch11/yahoo_data(SPY).csv')

In [47]:
# dta = data.get_data_yahoo('SPY', '2006/01/01')

In [48]:
dta.info, dta.columns

(<bound method DataFrame.info of       Unnamed: 0        Date        High         Low        Open       Close  \
 0              0  2006-01-03  127.000000  124.389999  125.190002  126.699997   
 1              1  2006-01-04  127.489998  126.699997  126.860001  127.300003   
 2              2  2006-01-05  127.589996  126.879997  127.150002  127.379997   
 3              3  2006-01-06  128.580002  127.360001  128.020004  128.440002   
 4              4  2006-01-09  129.059998  128.380005  128.419998  128.770004   
 ...          ...         ...         ...         ...         ...         ...   
 4201        4201  2022-09-12  411.730011  408.459991  408.779999  410.970001   
 4202        4202  2022-09-13  403.100006  391.920013  401.829987  393.100006   
 4203        4203  2022-09-14  396.200012  391.119995  394.470001  394.600006   
 4204        4204  2022-09-15  395.959991  388.779999  392.959991  390.119995   
 4205        4205  2022-09-16  385.879913  382.109985  384.140015  383.325012

In [49]:
dta.to_csv('../../../CSV Files/O_Reilly/ch11/yahoo_data(SPY).csv')

Now, we’ll compute daily returns and a function for transforming the returns into a trend signal formed from a lagged moving sum:

In [50]:
px = dta['Adj Close']

returns = px.pct_change()

In [51]:
def to_index(rets):
    index = (1 + rets).cumprod()
    first_loc = max(index.notnull().argmax() -1, 0)
    index.values[first_loc] = 1
    return index

In [52]:
def trend_signal(rets, lookback, lag):
    signal = rets.rolling(lookback, min_periods = lookback - 5).sum()
    return signal.shift(lag)

Using this function, we can (naively) create and test a trading strategy that trades this momentum signal  every Friday:

In [87]:
pd.to_datetime(signal)

0             NaT
1             NaT
2             NaT
3             NaT
4             NaT
          ...    
4201   1970-01-01
4202   1970-01-01
4203   1970-01-01
4204   1970-01-01
4205   1970-01-01
Name: Adj Close, Length: 4206, dtype: datetime64[ns]

In [53]:
signal = trend_signal(returns, 100, 3)

In [75]:
aa = pd.DataFrame(pd.to_datetime(signal))

bb = aa.dropna()

In [95]:

trade_friday = bb.resample('B')

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Int64Index'

In [55]:
trade_rets = trade_friday.shift(1) * returns

NameError: name 'trade_friday' is not defined

We can then convert the strategy returns to a return index an plot them:

In [ ]:
to_index(trade_rets).plot()

Suppose you wanted to decompose the strategy performance into more and less volatile periods of trading. Trailing one-year annualized standard deviation is a simple measure of volatility, and we can compute Sharpe ratios to assess the reward-to-risk ratio in various volatility regimes:

In [ ]:
vol = pd.rolling_std(returns, 250, min_periods = 200)* np.sqrt(250)

In [ ]:
def sharpe(rets, ann = 250):
    return rets.mean()/ rets.std() * np.sqrt(ann)

Now, dividing vol into quartiles with qcut and aggregating with sharpe we obtain:

In [ ]:
trade_rets.groupby(pd.qcut(vol, 4)).agg(sharpe)

These results show that the strategy performed the best during the period when the volatility was the hightest.